# Atari Breakout in RLlib

In [ ]:
#!pip install ray[rllib]
#!pip install ipywidgets gputil
#!pip install gym[atari] autorom[accept-rom-license] ray[rllib] torch torchvision atari-py ale-py ale_py
#!pip install "gym[accept-rom-license, atari]"

In [ ]:
import gym
import ray
from ray import tune
import ale_py



if ray.is_initialized():
    ray.shutdown()
ray.init(num_cpus=20, num_gpus=1)
    #env = gym.make("ALE/Breakout-v5")

checkpoint_directory = '/home/fitz/ray_results/PPO_2024-04-25_23-36-42/PPO_ALE_Breakout-v5_585c2_00000_0_2024-04-25_23-36-42/checkpoint_000184'
config = {
    "env": "ALE/Breakout-v5",
    "num_workers": 19,
    "num_envs_per_worker": 4,
    "num_gpus": 1,
    "framework": "torch",  
    "gamma": 0.99,
    "train_batch_size": 5000,
    "rollout_fragment_length": "auto",
    "sgd_minibatch_size": 500,
    "num_sgd_iter": 10,
    "clip_param": 0.1,
    "entropy_coeff": 0.01,
    "batch_mode": "truncate_episodes",
    "lambda": 0.95,
    "kl_coeff": 0.5,
    "clip_rewards": True,
    "clip_param": 0.1,
    "vf_clip_param": 10.0,
    "entropy_coeff": 0.01,
}


#config = {
#        "env": "ALE/Breakout-v5",
#        "preprocessor_pref": None,
#   #     "gamma": 0.99,
#  #      "num_gpus": 1,
# #       "num_workers": 19,
##        "num_envs_per_worker": 4,
#        "create_env_on_driver": True,
#        "lambda": 0.95,
#        "kl_coeff": 0.5,
#        "clip_rewards": True,
#        "clip_param": 0.1,
#        "vf_clip_param": 10.0,
#        "entropy_coeff": 0.01,
##        "rollout_fragment_length": "auto",
##        "sgd_minibatch_size": 500,
##        "num_sgd_iter": 10,
##        "batch_mode": "truncate_episodes",
##        "framework": "torch",
#}

stop_criteria = {
    "training_iteration": 15000,
    "episode_reward_mean": 200
}

results = tune.run(
    "PPO",
    config=config,
    stop=stop_criteria,
    checkpoint_at_end=True,
    checkpoint_freq=20,
    keep_checkpoints_num=20,  # Keep 10 checkpoints
    checkpoint_score_attr="episode_reward_mean",  # Retain checkpoints with the highest episode rewards
    restore=checkpoint_directory
)
    


2024-04-26 08:58:58,218	INFO worker.py:1749 -- Started a local Ray instance.
2024-04-26 08:58:58,805	INFO tune.py:624 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(RolloutWorker pid=2682418) A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
(RolloutWorker pid=2682418) [Powered by Stella]
2024-04-26 08:59:12,222	WARNING logger.py:186 -- Remote file not found: /home/fitz/ray_results/PPO_2024-04-26_08-58-58/PPO_ALE_Breakout-v5_e4855_00000_0_2024-04-26_08-58-58/result.json
(PPO pid=2682269) Restored on 192.168.1.114 from checkpoint: Checkpoint(filesystem=local, path=/home/fitz/ray_results/PPO_2024-04-25_23-36-42/PPO_ALE_Breakout-v5_585c2_00000_0_2024-04-25_23-36-42/checkpoint_000184)
(PPO pid=2682269) 2024-04-26 08:59:18,642	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!
(PPO pid=2682269) A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7) [repeated 19x across cluster]
(PPO pid=2682269) [Powered by Stella] [repeated 19x across cluster]


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_timesteps_total,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_ALE_Breakout-v5_e4855_00000,41005000,{},"{'num_env_steps_sampled': 41005000, 'num_env_steps_trained': 41005000, 'num_agent_steps_sampled': 41005000, 'num_agent_steps_trained': 41005000}",{},nan,{},nan,nan,nan,0,0,"{'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.0, 'cur_lr': 5e-05, 'total_loss': -0.01103209837092436, 'policy_loss': -0.007179453528951853, 'vf_loss': 0.004520092289312743, 'vf_explained_var': 0.9954019129276276, 'kl': 0.0043703861951019, 'entropy': 0.8372739440202713, 'entropy_coeff': 0.009999999999999998}, 'model': {}, 'num_grad_updates_lifetime': 50.5, 'diff_num_grad_updates_vs_sampler_policy': 49.5}}, 'num_env_steps_sampled': 41005000, 'num_env_steps_trained': 41005000, 'num_agent_steps_sampled': 41005000, 'num_agent_steps_trained': 41005000}",41005000,41005000,41005000,5000,539.422,41005000,5000,539.422,0,19,0,0,5000,"{'cpu_util_percent': 61.61538461538463, 'ram_util_percent': 21.00769230769231, 'gpu_util_percent0': 0.2876923076923077, 'vram_util_percent0': 0.18840895432692306}",{},{},{},{},"{'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'episodes_timesteps_total': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0, 'connector_metrics': {}}","{'training_iteration_time_ms': 9269.195, 'restore_workers_time_ms': 0.017, 'training_step_time_ms': 9269.148, 'sample_time_ms': 6348.208, 'load_time_ms': 100.959, 'load_throughput': 49525.258, 'learn_time_ms': 2786.206, 'learn_throughput': 1794.555, 'synch_weights_time_ms': 30.868}"


2024-04-26 08:59:21,571	WARNING logger.py:186 -- Remote file not found: /home/fitz/ray_results/PPO_2024-04-26_08-58-58/PPO_ALE_Breakout-v5_e4855_00000_0_2024-04-26_08-58-58/progress.csv


In [8]:
best_trial = results.get_best_trial("episode_reward_mean", "max", "last")
best_checkpoint = results.get_best_checkpoint(best_trial, "episode_reward_mean", "max")
print("Best checkpoint:", best_checkpoint)


Best checkpoint: Checkpoint(filesystem=local, path=/home/fitz/ray_results/PPO_2024-04-25_23-36-42/PPO_ALE_Breakout-v5_585c2_00000_0_2024-04-25_23-36-42/checkpoint_000184)


In [ ]:
ray.shutdown()